<a href="https://colab.research.google.com/github/mrasw/Big-Data/blob/main/EAS_Company_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!wget https://bayueas-114.s3.amazonaws.com/data-source/classification-dataset-v1.csv -O classification-dataset-v1.csv

--2022-06-08 15:31:07--  https://bayueas-114.s3.amazonaws.com/data-source/classification-dataset-v1.csv
Resolving bayueas-114.s3.amazonaws.com (bayueas-114.s3.amazonaws.com)... 52.217.204.209
Connecting to bayueas-114.s3.amazonaws.com (bayueas-114.s3.amazonaws.com)|52.217.204.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 399850400 (381M) [text/csv]
Saving to: ‘classification-dataset-v1.csv’

classification-data 100%[===================>] 381.33M  84.1MB/s    in 4.8s    

2022-06-08 15:31:12 (79.8 MB/s) - ‘classification-dataset-v1.csv’ saved [399850400/399850400]



In [22]:
# Install Apache Spark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 42.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=dba5c3f5fbc93f1d6c5dabe17f30d066410fa29668412a2dc15bce9dd5497538
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [28]:
from pyspark.sql import SparkSession


spark = SparkSession.builder.appName('Clustering using K-Means').getOrCreate()


In [44]:
df=spark.read.option("header", True).csv("classification-dataset-v1.csv")

df.printSchema()

root
 |-- Category: string (nullable = true)
 |-- website: string (nullable = true)
 |-- company_name: string (nullable = true)
 |-- homepage_text: string (nullable = true)
 |-- h1: string (nullable = true)
 |-- h2: string (nullable = true)
 |-- h3: string (nullable = true)
 |-- nav_link_text: string (nullable = true)
 |-- meta_keywords: string (nullable = true)
 |-- meta_description: string (nullable = true)



In [45]:
df.head()
df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|            Category|             website|        company_name|       homepage_text|                  h1|                  h2|                  h3|       nav_link_text|       meta_keywords|    meta_description|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Commercial Servic...|     bipelectric.com|bip dipietro elec...|                 ...|                null|                null|                null|                null|electricians vero...|                null|
| vero beach electric| commercial light...|         electrician| Indian River County|           Sebastian|          Ft Pierce"|Providing quality...|    

In [46]:
print((df.count(), len(df.columns)))

(250419, 10)


In [89]:
col_null_cnt_df =  df.select([count(when(col(c).isNull(),c)).alias(c) for c in df.columns])

display(col_null_cnt_df)


DataFrame[Category: bigint, website: bigint, company_name: bigint, homepage_text: bigint, h1: bigint, h2: bigint, h3: bigint, nav_link_text: bigint, meta_keywords: bigint, meta_description: bigint]

In [70]:
from pyspark.sql import functions as F
df.where(F.col("Category").isNull()).show()

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Category|             website|        company_name|       homepage_text|                  h1|                  h2|                  h3|       nav_link_text|       meta_keywords|    meta_description|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|    null|    blue chip stocks|tax saving mutual...|          debt funds|     sip mutual fund|best investment o...|best performing m...|systematic invest...|    best investments|best mutual funds...|
|    null|         elss scheme|mutual fund compa...|      top elss funds| sip investment plan| tax on mutual funds|best equity mutua...|   top 10 elss funds|high return mutua...|top mutual funds ...|


In [58]:
df.where(F.col("website").isNull()).show()

+--------------------+-------+--------------------+-------------+--------------------+----+----+-------------+-------------+----------------+
|            Category|website|        company_name|homepage_text|                  h1|  h2|  h3|nav_link_text|meta_keywords|meta_description|
+--------------------+-------+--------------------+-------------+--------------------+----+----+-------------+-------------+----------------+
|ET EXPERTISE#sep#...|   null|,en,à propos,nos ...|         null|TRIOX Urgences En...|null|null|         null|         null|            null|
|big and small.#se...|   null|                null|         null|                null|null|null|         null|         null|            null|
|in salt lake for ...|   null|                null|         null|                null|null|null|         null|         null|            null|
|  doesnât stop ...|   null|                null|         null|                null|null|null|         null|         null|            null|
|  the

In [59]:
df.where(F.col("company_name").isNull()).show()

+--------------------+--------------------+------------+--------------------+--------------------+----+----+-------------+-------------+----------------+
|            Category|             website|company_name|       homepage_text|                  h1|  h2|  h3|nav_link_text|meta_keywords|meta_description|
+--------------------+--------------------+------------+--------------------+--------------------+----+----+-------------+-------------+----------------+
|       661.663.0300"|Get To Know Elias...|        null|Elias Medical bak...|For the best valu...|null|null|         null|         null|            null|
|    Locally owned...| we have everythi...|        null|                null|                null|null|null|         null|         null|            null|
|             Anytime| anywhere.#sep#Ca...|        null|                null|                null|null|null|         null|         null|            null|
|big and small.#se...|                null|        null|                null

In [60]:
df.where(F.col("homepage_text").isNull()).show()

+--------------------+--------------------+--------------------+-------------+--------------------+----+----+-------------+-------------+----------------+
|            Category|             website|        company_name|homepage_text|                  h1|  h2|  h3|nav_link_text|meta_keywords|meta_description|
+--------------------+--------------------+--------------------+-------------+--------------------+----+----+-------------+-------------+----------------+
|661.663.0300   Fa...| Suite 203 Bakers...| CA 93311 | (phon...|         null|                null|null|null|         null|         null|            null|
|    (fax) 661.663...|Offering Bakersfi...|Welcome to ELIAS ...|         null|                null|null|null|         null|         null|            null|
|ET EXPERTISE#sep#...|                null|,en,à propos,nos ...|         null|TRIOX Urgences En...|null|null|         null|         null|            null|
|    Locally owned...| we have everythi...|                null|      

In [61]:
df.where(F.col("h1").isNull()).show()

+--------------------+--------------------+--------------------+--------------------+----+--------------------+--------------------+--------------------+--------------------+--------------------+
|            Category|             website|        company_name|       homepage_text|  h1|                  h2|                  h3|       nav_link_text|       meta_keywords|    meta_description|
+--------------------+--------------------+--------------------+--------------------+----+--------------------+--------------------+--------------------+--------------------+--------------------+
|Commercial Servic...|     bipelectric.com|bip dipietro elec...|                 ...|null|                null|                null|                null|electricians vero...|                null|
|          Healthcare|    eliasmedical.com|       elias medical|    site map | en...|null|                null|                null|                null|                null|                null|
|661.663.0300   Fa..

In [62]:
df.where(F.col("h2").isNull()).show()

+--------------------+--------------------+--------------------+--------------------+--------------------+----+--------------------+--------------------+--------------------+--------------------+
|            Category|             website|        company_name|       homepage_text|                  h1|  h2|                  h3|       nav_link_text|       meta_keywords|    meta_description|
+--------------------+--------------------+--------------------+--------------------+--------------------+----+--------------------+--------------------+--------------------+--------------------+
|Commercial Servic...|     bipelectric.com|bip dipietro elec...|                 ...|                null|null|                null|                null|electricians vero...|                null|
|          Healthcare|    eliasmedical.com|       elias medical|    site map | en...|                null|null|                null|                null|                null|                null|
|661.663.0300   Fa..

In [63]:
df.where(F.col("h3").isNull()).show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----+-------------+--------------------+--------------------+
|            Category|             website|        company_name|       homepage_text|                  h1|                  h2|  h3|nav_link_text|       meta_keywords|    meta_description|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----+-------------+--------------------+--------------------+
|Commercial Servic...|     bipelectric.com|bip dipietro elec...|                 ...|                null|                null|null|         null|electricians vero...|                null|
|          Healthcare|    eliasmedical.com|       elias medical|    site map | en...|                null|                null|null|         null|                null|                null|
|661.663.0300   Fa...| Suite 203 Bakers...| CA 93311 | 

In [65]:
df.where(F.col("nav_link_text").isNull()).show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+
|            Category|             website|        company_name|       homepage_text|                  h1|                  h2|                  h3|nav_link_text|       meta_keywords|    meta_description|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+
|Commercial Servic...|     bipelectric.com|bip dipietro elec...|                 ...|                null|                null|                null|         null|electricians vero...|                null|
| vero beach electric| commercial light...|         electrician| Indian River County|           Sebastian|          Ft Pierce"|Providing quality...|         null|                nu

In [66]:
df.where(F.col("meta_keywords").isNull()).show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+
|            Category|             website|        company_name|       homepage_text|                  h1|                  h2|                  h3|       nav_link_text|meta_keywords|    meta_description|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+
| vero beach electric| commercial light...|         electrician| Indian River County|           Sebastian|          Ft Pierce"|Providing quality...|                null|         null|                null|
|          Healthcare|    eliasmedical.com|       elias medical|    site map | en...|                null|                null|                null|                null|         nu

In [67]:
df.where(F.col("meta_description").isNull()).show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+
|            Category|             website|        company_name|       homepage_text|                  h1|                  h2|                  h3|       nav_link_text|       meta_keywords|meta_description|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+
|Commercial Servic...|     bipelectric.com|bip dipietro elec...|                 ...|                null|                null|                null|                null|electricians vero...|            null|
| vero beach electric| commercial light...|         electrician| Indian River County|           Sebastian|          Ft Pierce"|Providing quality...|                null

membuat kolom baru dari gabungan h1 h2 h3 menjadi heading



In [109]:
# concat_ws(",","firstname",'lastname'
new_df = df.withColumn("headings",concat_ws(" ","h1",'h2',"h3"))
new_df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|            Category|             website|        company_name|       homepage_text|                  h1|                  h2|                  h3|       nav_link_text|       meta_keywords|    meta_description|            headings|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Commercial Servic...|     bipelectric.com|bip dipietro elec...|                 ...|                null|                null|                null|                null|electricians vero...|                null|                    |
| vero beach electric| commercial light...|         electrician| Ind

drop h1 h2 h3

In [122]:
cols = ("h1","h2","h3")
new_df.drop(*cols).printSchema()
new_df.drop(*cols).show()

root
 |-- Category: string (nullable = true)
 |-- website: string (nullable = true)
 |-- company_name: string (nullable = true)
 |-- homepage_text: string (nullable = true)
 |-- nav_link_text: string (nullable = true)
 |-- meta_keywords: string (nullable = true)
 |-- meta_description: string (nullable = true)
 |-- headings: string (nullable = false)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|            Category|             website|        company_name|       homepage_text|       nav_link_text|       meta_keywords|    meta_description|            headings|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Commercial Servic...|     bipelectric.com|bip dipietro elec...|                 ...|                null|electricians vero...|          